# Chapter 15: The Complexity of Sparse Recovery

In [1]:
import numpy as np
from numpy import linalg as LA
from scipy import linalg

## Prony method: s-sparse recovery from 2s Fourier observations

In [2]:
# generate an s-sparse vector together with its first 2s Fourier observations
N = 500               # ambient dimension
s = 10                # sparsity level
m = 2*s               # number of observations
# create the sparse vector x
x = np.zeros(N)
supp_origi = np.sort(np.random.permutation(N)[:s])
x[supp_origi] = np.random.normal(0,1,s)
# produce the observation vector y made of exact m=2s Fourier coefficients 
xhat = np.fft.fft(x)
y_exact = xhat[:m]

In [3]:
# Exact recovery of the support of x via Prony method
phat = np.zeros(N,dtype=np.cfloat)
phat[0] = 1
M = linalg.toeplitz(y_exact[s-1:2*s-1], y_exact[s-1::-1])
phat[1:s+1] = -linalg.solve(M,y_exact[s:2*s])
p = np.fft.ifft(phat)
idx = np.argsort(abs(p))
supp_exact = np.sort(idx[0:s])
print('In the exact case, the original and recovered supports agree:')
print(supp_origi)
print(supp_exact)

In the exact case, the original and recovered supports agree:
[133 166 206 211 239 245 311 361 384 435]
[133 166 206 211 239 245 311 361 384 435]


## Nonrobustness (hence nonstability) of Prony method

In [4]:
# Adding observation noise destroys the recovery of the support
noise = 1e-5*np.random.normal(0,1,m)
y_noisy = y_exact+noise
phat = np.zeros(N,dtype=np.cfloat)
phat[0] = 1
M = linalg.toeplitz(y_noisy[s-1:2*s-1], y_noisy[s-1::-1])
phat[1:s+1] = -linalg.solve(M,y_noisy[s:2*s])
p = np.fft.ifft(phat)
idx = np.argsort(abs(p))
supp_noisy = np.sort(idx[0:s])
print('In the noisy case, the original and recovered supports do not agree anymore:')
print(supp_origi)
print(supp_noisy)

In the noisy case, the original and recovered supports do not agree anymore:
[133 166 206 211 239 245 311 361 384 435]
[133 166 208 209 242 243 311 361 384 435]


In [5]:
# The outputted vectors do not agree either
F = linalg.dft(N)           # the full discrete Fourier matrix
A = F[:2*s,:]               # the submatrix for the first 2s Fourier coefficients
x_exact = np.zeros(N, dtype=np.cfloat)
x_exact[supp_exact] = linalg.solve(np.transpose(A[:,supp_exact])@A[:,supp_exact],
                                   np.transpose(A[:,supp_exact])@y_exact)
rel_error_exact = LA.norm(x-x_exact)/LA.norm(x)
x_noisy = np.zeros(N, dtype=np.cfloat)
x_noisy[supp_noisy] = linalg.solve(np.transpose(A[:,supp_noisy])@A[:,supp_noisy],
                                   np.transpose(A[:,supp_noisy])@y_noisy)
rel_error_noisy = LA.norm(x-x_noisy)/LA.norm(x)
print('Recovery from exact observations is quite successful: relative L2-error = {:.2e}.'.format(rel_error_exact))
print('Recovery from inexact observations is not successful: relative L2-error = {:.2e}.'.format(rel_error_noisy))

Recovery from exact observations is quite successful: relative L2-error = 7.65e-13.
Recovery from inexact observations is not successful: relative L2-error = 9.32e-01.


## Robustness of $\ell_1$-minimization 

Note that the number of observations need to be increased: Chapter 15 says that stability and robustness cannot be achieved with $m=2s$ observations.

In [6]:
import cvxpy as cp

In [7]:
# For L1 minimization, adding observation noise does not destroy the recovery 
m = 4*s
A = F[:m,:] 
y_exact = A@x
noise = 1e-5*np.random.normal(0,1,m)
y_noisy = y_exact+noise
x1_exact = cp.Variable(N)
objective = cp.Minimize(cp.norm(x1_exact,1))
constraints = [ A@x1_exact == y_exact]
exact = cp.Problem(objective,constraints)
exact.solve(solver='SCS',max_iters=3000,eps=1e-6)
x1_exact = x1_exact.value
rel_error_exact = LA.norm(x-x1_exact)/LA.norm(x)
print('Recovery from exact observations is quite successful: relative L2-error = {:.2e}.'.format(rel_error_exact))
x1_noisy = cp.Variable(N)
objective = cp.Minimize(cp.norm(x1_noisy,1))
constraints = [A@x1_noisy == y_noisy]
noisy = cp.Problem(objective,constraints)
noisy.solve(solver='SCS',max_iters=3000,eps=1e-6)
x1_noisy = x1_noisy.value
rel_error_noisy = LA.norm(x-x1_noisy)/LA.norm(x)
print('Recovery from inexact observations is not bad either: relative L2-error = {:.2e}.'.format(rel_error_noisy))

Recovery from exact observations is quite successful: relative L2-error = 2.23e-06.
Recovery from inexact observations is not bad either: relative L2-error = 2.80e-06.
